In [1]:
# Importing necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Step 1: Load and preprocess the data
def load_and_preprocess_data():
    # Load the California Housing dataset
    cali = fetch_california_housing()
    X = cali.data  # Features
    y = cali.target  # Labels (House Prices)

    # Splitting data into train and test sets (80% train, 20% test)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Standardizing the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Converting to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)  # Reshaping to (n_samples, 1)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

    return X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor

# Step 2: Define the neural network model
class CaliforniaNet(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size=1):
        super(CaliforniaNet, self).__init__()
        # First hidden layer
        self.fc1 = nn.Linear(input_size, hidden_size1)
        # Second hidden layer
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        # Output layer
        self.fc3 = nn.Linear(hidden_size2, output_size)
    
    def forward(self, x):
        # Apply ReLU activations after each layer except the last
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        # Output layer does not apply any activation function
        x = self.fc3(x)
        return x

# Step 3: Train the model
def train_model(model, X_train, y_train, epochs=100, learning_rate=0.01):
    # Loss function (Mean Squared Error for regression)
    criterion = nn.MSELoss()
    # Optimizer (Adam)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(X_train)
        # Calculate the loss
        loss = criterion(outputs, y_train)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Step 4: Evaluate the model
def evaluate_model(model, X_test, y_test):
    with torch.no_grad():  # Disable gradient calculation
        outputs = model(X_test)
        mse_loss = nn.MSELoss()(outputs, y_test)
        print(f'Test Mean Squared Error: {mse_loss.item():.4f}')

# Step 5: Putting it all together
if __name__ == "__main__":
    # Load data
    X_train, X_test, y_train, y_test = load_and_preprocess_data()

    # Define model
    input_size = X_train.shape[1]
    hidden_size1 = 64
    hidden_size2 = 32
    model = CaliforniaNet(input_size, hidden_size1, hidden_size2)

    # Train model
    train_model(model, X_train, y_train, epochs=100, learning_rate=0.01)

    # Evaluate model
    evaluate_model(model, X_test, y_test)


/data/aulas/.venv/lib/python3.12/site-packages/sklearn/datasets/_base.py:1472: UserWarning: Retry downloading from url: https://ndownloader.figshare.com/files/5976036
  warnings.warn(f"Retry downloading from url: {remote.url}")


URLError: <urlopen error [Errno -3] Temporary failure in name resolution>